In [24]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress=True)

In [102]:
df = pd.read_csv('Sweden Load Data 2005-2017.csv')
df.rename({'cet_cest_timestamp':'time','SE_load_actual_tso':'load'},axis='columns',inplace=True)
df['time'] = pd.to_datetime(df['time'],errors='ignore', utc=True)

In [104]:
df['weekday'] = df['time'].dt.weekday

In [120]:
df['load'].values.tolist()

[15991.34,
 15673.12,
 15431.81,
 15326.48,
 15468.59,
 15717.19,
 15936.22,
 16032.7,
 16091.51,
 16563.9,
 16997.13,
 17120.22,
 17237.21,
 17418.9,
 17939.93,
 18537.65,
 18610.38,
 18302.57,
 17935.56,
 17465.78,
 16950.03,
 16335.14,
 15708.7,
 15107.03,
 14794.95,
 14685.88,
 14615.4,
 14617.3,
 14813.46,
 15158.68,
 15683.33,
 16166.35,
 16655.89,
 17256.68,
 17658.23,
 17704.97,
 17849.45,
 18066.68,
 18691.28,
 19334.58,
 19453.6,
 19236.18,
 18961.63,
 18644.97,
 18021.66,
 17317.62,
 16589.04,
 16050.19,
 15792.37,
 15706.93,
 15741.03,
 16104.5,
 16911.38,
 18586.34,
 20342.09,
 21078.03,
 21078.13,
 21212.16,
 21212.42,
 21095.02,
 20995.86,
 21124.35,
 21708.93,
 22119.4,
 22046.0,
 21645.52,
 21183.17,
 20463.09,
 19667.84,
 18705.46,
 17649.67,
 16874.03,
 16509.31,
 16385.98,
 16327.34,
 16583.86,
 17265.29,
 18875.05,
 20650.26,
 21299.2,
 21161.54,
 21345.62,
 21427.29,
 21430.74,
 21457.98,
 21573.71,
 22176.52,
 22153.98,
 21913.14,
 21309.24,
 20732.16,
 20084.68,

In [139]:
test_df = df.head(25).copy()

In [140]:
test_df['flattened_load'] = test_df['load'].values.tolist()
test_df['action'] = 0

In [141]:
test_df.iloc[0,-1] = 1

In [145]:
test_df['action_shifted'] = test_df['action'].shift()
test_df['flattened_load'] = test_df['flattened_load'].shift()

In [246]:
test_df.weekday.

,time,load,weekday,flattened_load,action,action_shifted
0,2005-01-01 00:00:00+00:00,15991.34,5,nan,1,nan
1,2005-01-01 01:00:00+00:00,15673.12,5,15991.34,0,1.00
2,2005-01-01 02:00:00+00:00,15431.81,5,15673.12,0,0.00
3,2005-01-01 03:00:00+00:00,15326.48,5,15431.81,0,0.00
4,2005-01-01 04:00:00+00:00,15468.59,5,15326.48,0,0.00
5,2005-01-01 05:00:00+00:00,15717.19,5,15468.59,0,0.00
6,2005-01-01 06:00:00+00:00,15936.22,5,15717.19,0,0.00
7,2005-01-01 07:00:00+00:00,16032.70,5,15936.22,0,0.00
8,2005-01-01 08:00:00+00:00,16091.51,5,16032.70,0,0.00
9,2005-01-01 09:00:00+00:00,16563.90,5,16091.51,0,0.00


In [151]:
test_df.load

0    15991.34
1    15673.12
2    15431.81
3    15326.48
4    15468.59
5    15717.19
6    15936.22
7    16032.70
8    16091.51
9    16563.90
10   16997.13
11   17120.22
12   17237.21
13   17418.90
14   17939.93
15   18537.65
16   18610.38
17   18302.57
18   17935.56
19   17465.78
20   16950.03
21   16335.14
22   15708.70
23   15107.03
24   14794.95
Name: load, dtype: float64

In [154]:
test_df.load.values

array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
       15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
       17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
       17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
       14794.95])

In [113]:
class Complex:
     def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

x = Complex(3.0)

TypeError: __init__() missing 1 required positional argument: 'imagpart'

In [245]:
test_df['load']

0    15991.34
1    15673.12
2    15431.81
3    15326.48
4    15468.59
5    15717.19
6    15936.22
7    16032.70
8    16091.51
9    16563.90
10   16997.13
11   17120.22
12   17237.21
13   17418.90
14   17939.93
15   18537.65
16   18610.38
17   18302.57
18   17935.56
19   17465.78
20   16950.03
21   16335.14
22   15708.70
23   15107.03
24   14794.95
Name: load, dtype: float64

In [284]:
class Agent:    
    def __init__(self,array_loads,array_weekdays,load_idx=0,charge=4,action_space=np.array(['charge','discharge','wait']),policy='random'):
        #load whether we charge
        self.array_loads = array_loads
        self.array_weekdays=array_weekdays
        self.load_idx = load_idx
        self.day_idx = load_idx
        self.charge = charge
        self.policy = policy
        self.prev_state=None
        self.collection_loads = []
        self.collection_rewards = []
    def generate_curr_state(self):
        self.curr_state = self.array_loads[self.load_idx]
        self.collection_loads.append(self.curr_state)
        return self.curr_state
    
    def take_action(self):
        #check whether policy is random and assign candidate action
        if self.policy=='random':
            candidate_action = np.random.choice(action_space, 1)[0]
            #print(candidate_action)
            if candidate_action == 'discharge' and self.charge!=0:
                self.load_idx+=1
                self.day_idx+=1
                if self.array_loads[self.load_idx] >= self.curr_state:
                    self.collection_loads.append(self.curr_state)
                else:
                    self.curr_state = self.generate_curr_state()
                self.charge-=1
            elif candidate_action == 'charge' and self.charge!=4:
                self.load_idx+=1
                self.day_idx+=1
                if self.array_loads[self.load_idx] <= self.curr_state:
                    self.collection_loads.append(self.curr_state)
                else:
                    self.curr_state = self.generate_curr_state()
                self.charge+=1
            else:
                self.load_idx+=1
                self.day_idx+=1
                self.curr_state = self.generate_curr_state()
            self.prev_state= self.collection_loads[self.load_idx-1]

        if self.array_weekdays[self.day_idx]== self.array_weekdays[self.day_idx-1]:
            self.collection_rewards.append(0)
        else:
            cost = 5+0.5*max(self.collection_loads[:-1])+0.05*(max(self.collection_loads[:-1])**2)
            self.collection_rewards.append(cost)
        self.curr_reward = self.collection_rewards[-1]
        #return (self.prev_state,candidate_action,self.curr_reward,self.curr_state)
    def clear_collection_loads(self):
        self.collection_loads = []

create agent

In [255]:
agent1 = Agent(test_df.load.values,test_df.weekday.values)
agent1.generate_curr_state()

generate current state

In [256]:
agent1.generate_curr_state()

15991.34

In [257]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 0,
 'day_idx': 0,
 'charge': 4,
 'policy': 'random',
 'prev_state': None,
 'collection_loads': [15991.34],
 'collection_rewards': [],
 'curr_state': 15991.34}

First action

In [258]:
agent1.take_action()

wait


(15991.34, 'wait', 0, 15673.12)

In [259]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 1,
 'day_idx': 1,
 'charge': 4,
 'policy': 'random',
 'prev_state': 15991.34,
 'collection_loads': [15991.34, 15673.12],
 'collection_rewards': [0],
 'curr_state': 15673.12,
 'curr_reward': 0}

Second action

In [260]:
agent1.take_action()

wait


(15673.12, 'wait', 0, 15431.81)

In [261]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 2,
 'day_idx': 2,
 'charge': 4,
 'policy': 'random',
 'prev_state': 15673.12,
 'collection_loads': [15991.34, 15673.12, 15431.81],
 'collection_rewards': [0, 0],
 'curr_state': 15431.81,
 'curr_reward': 0}

third action

In [262]:
agent1.take_action()

wait


(15431.81, 'wait', 0, 15326.48)

In [263]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 3,
 'day_idx': 3,
 'charge': 4,
 'policy': 'random',
 'prev_state': 15431.81,
 'collection_loads': [15991.34, 15673.12, 15431.81, 15326.48],
 'collection_rewards': [0, 0, 0],
 'curr_state': 15326.48,
 'curr_reward': 0}

In [34]:
#action_space = np.array(['charge','discharge','wait'])

In [264]:
agent1.take_action()

charge


(15326.48, 'charge', 0, 15468.59)

In [265]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 4,
 'day_idx': 4,
 'charge': 4,
 'policy': 'random',
 'prev_state': 15326.48,
 'collection_loads': [15991.34, 15673.12, 15431.81, 15326.48, 15468.59],
 'collection_rewards': [0, 0, 0, 0],
 'curr_state': 15468.59,
 'curr_reward': 0}

In [266]:
agent1.take_action()

wait


(15468.59, 'wait', 0, 15717.19)

In [268]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 5,
 'day_idx': 5,
 'charge': 4,
 'policy': 'random',
 'prev_state': 15468.59,
 'collection_loads': [15991.34,
  15673.12,
  15431.81,
  15326.48,
  15468.59,
  15717.19],
 'collection_rewards': [0, 0, 0, 0, 0],
 'curr_state': 15717.19,
 'curr_reward': 0}

In [269]:
for i in range(10):
    agent1.take_action()

wait
charge
discharge
discharge
charge
charge
charge
discharge
charge
discharge


In [270]:
agent1.__dict__

{'array_loads': array([15991.34, 15673.12, 15431.81, 15326.48, 15468.59, 15717.19,
        15936.22, 16032.7 , 16091.51, 16563.9 , 16997.13, 17120.22,
        17237.21, 17418.9 , 17939.93, 18537.65, 18610.38, 18302.57,
        17935.56, 17465.78, 16950.03, 16335.14, 15708.7 , 15107.03,
        14794.95]),
 'array_weekdays': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6]),
 'load_idx': 15,
 'day_idx': 15,
 'charge': 3,
 'policy': 'random',
 'prev_state': 17939.93,
 'collection_loads': [15991.34,
  15673.12,
  15431.81,
  15326.48,
  15468.59,
  15717.19,
  15936.22,
  16032.7,
  16032.7,
  16032.7,
  16997.13,
  17120.22,
  17237.21,
  17237.21,
  17939.93,
  17939.93],
 'collection_rewards': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'curr_state': 17939.93,
 'curr_reward': 0}

In [271]:
agent1.take_action()

discharge


(17939.93, 'discharge', 0, 17939.93)

In [272]:
agent1.take_action()

charge


(17939.93, 'charge', 0, 18302.57)

In [273]:
agent1.take_action()

discharge


(18302.57, 'discharge', 0, 17935.56)

In [274]:
agent1.take_action()

wait


(17935.56, 'wait', 0, 17465.78)

In [275]:
agent1.take_action()

wait


(17465.78, 'wait', 0, 16950.03)

In [276]:
agent1.take_action()

wait


(16950.03, 'wait', 0, 16335.14)

In [277]:
agent1.take_action()

charge


(16335.14, 'charge', 0, 16335.14)

In [278]:
agent1.take_action()

discharge


(16335.14, 'discharge', 0, 15107.03)

In [279]:
agent1.take_action()

discharge


(15107.03, 'discharge', 16758359.715245001, 14794.95)

In [280]:
agent1.take_action()

discharge


IndexError: index 25 is out of bounds for axis 0 with size 25

In [281]:
agent1.collection_rewards

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 16758359.715245001]

In [283]:
df

,time,load,weekday
0,2005-01-01 00:00:00+00:00,15991.34,5
1,2005-01-01 01:00:00+00:00,15673.12,5
2,2005-01-01 02:00:00+00:00,15431.81,5
3,2005-01-01 03:00:00+00:00,15326.48,5
4,2005-01-01 04:00:00+00:00,15468.59,5


In [287]:
agent2 = Agent(df.load.values,df.weekday.values)
agent2.generate_curr_state()
for i in range(len(df)-1):
    agent2.take_action()

In [289]:
print(np.mean(agent2.collection_rewards))

1378453.0114575604


In [290]:
agent3 = Agent(df.load.values,df.weekday.values)
agent3.generate_curr_state()
for i in range(len(df)-1):
    agent3.take_action()
print(np.mean(agent3.collection_rewards))

1380690.8958933547
